Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Fcharges-193353%2Findex.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/charges-193353)]*

## [``dyce``](https://posita.github.io/dyce/) translation of [Hypergardens’ solution to “Modelling ‘rolling d6 >= charges left expends a charge’ mechanics with anydice”](https://rpg.stackexchange.com/a/195107/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
try:
    import anydyce
    import dyce
    import ipywidgets
except ImportError:
    requirements = [
        "anydyce>=0.1.4,<0.2",
        "dyce>=0.5.2,<0.6",
        "ipywidgets",
    ]
    try:
        import piplite
        await piplite.install(requirements)
    except ImportError:
        import pip
        pip.main(["install"] + requirements)

In [2]:
from dyce import H
from fractions import Fraction
import sys

try:
    from functools import cache
except ImportError:
    from functools import lru_cache
    cache = lru_cache(maxsize=None)

def rolls_before_depleting_one_charge(charges: int, d: H) -> H:
    return d.lt(charges).explode(
        max_depth=-1,  # <-- don't halt recusion based on depth ...
        precision_limit=Fraction(1, 250),  # <-- ... only on contextual precision
    )

@cache  # <-- probably not helpful until charges is pretty large (see below), but doesn't hurt
def expected_uses(charges: int, d: H = H(6)) -> H:
    return (
        H({0: 1}) if charges <= 0
        else 1 + rolls_before_depleting_one_charge(charges, d) + expected_uses(charges - 1)
    )

In [ ]:
from anydyce import BreakoutType, jupyter_visualize

d = 6
jupyter_visualize(
    [
        (f"expected uses on\nd{d} with {n} charges", expected_uses(n, H(d)))
        for n in range(0, d + 1)
    ],
    default_breakout_type=BreakoutType.BURST,
)

In [4]:
%%timeit

def expected_uses_not_cached(charges: int, d: H) -> H:
    return H({0: 1}) if charges <= 0 else 1 + rolls_before_depleting_one_charge(charges, d) + expected_uses_not_cached(charges - 1, d)

[expected_uses_not_cached(n, H(6)) for n in range(50, 0, -1)]

327 ms ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
@cache
def expected_uses_cached(charges: int, d: H) -> H:
    return H({0: 1}) if charges <= 0 else 1 + rolls_before_depleting_one_charge(charges, d) + expected_uses_cached(charges - 1, d)

[expected_uses_cached(n, H(6)) for n in range(50, 0, -1)]

11.8 ms ± 896 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
